In [248]:
from scipy.io import arff
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix,accuracy_score

data,meta = arff.loadarff("PhishingData.arff")

#meta
data_ = data.astype(np.dtype([('SFH', 'int64'), ('popUpWidnow', 'int64'), ('SSLfinal_State', 'int64'), ('Request_URL', 'int64'), ('URL_of_Anchor', 'int64'), ('web_traffic', 'int64'), ('URL_Length', 'int64'), ('age_of_domain', 'int64'), ('having_IP_Address', 'int64'), ('Result', 'int64')]))

count_1 = 0; count_neg_1 = 0; count_0 = 0


partition_by_classes = {
    -1: [],
    0:[],
    1: []
}
neg_class = [point for point in data_ if point[-1] == -1]
zero_class = [point for point in data_ if point[-1] == 0]
one_class = [point for point in data_ if point[-1] == 1]

# for point in data_:
#     if point[-1] == 1:
#        count_1 += 1
#     elif point[-1] == -1:
#         count_neg_1 +=1
#     else:
#         count_0 +=1
# print(count_neg_1, count_0, count_1)
#print(len(neg_class), len(zero_class), len(one_class))

k = 10

def create_k_fold(data, k):
    k_subset = []
    start_point = 0
    for i in range(k):
        if i == k -1:
            k_subset.append(data[start_point:])
            continue
        rough_size = len(data) //k;
        subset = data[start_point:start_point + rough_size]
        k_subset.append(subset)
        start_point = start_point + rough_size
    
    return k_subset

k_neg_class = create_k_fold(neg_class, 10)
k_zero_class = create_k_fold(zero_class, 10)
k_one_class = create_k_fold(one_class, 10)


In [251]:
def SVM(k_one_class,k_neg_class,k_zero_class,n_fold):
    ac = 0
    for i in range(n_fold):
        testing_index = i
        training_neg_class = k_neg_class[0:i+1] + k_neg_class[i+1:]
        training_zero_class =k_zero_class[0:i+1] + k_zero_class[i+1:]
        training_one_class = k_one_class[0:i+1] + k_one_class[i+1:]
        testing_neg_class = k_neg_class[i]
        testing_zero_class = k_zero_class[i]
        testing_one_class = k_one_class[i]
        train = training_neg_class + training_zero_class + training_one_class
        test = testing_neg_class +testing_zero_class+testing_one_class
        training_samples = []
        trainging_labels = []
        testing = []
        testing_labels = []


        for subset in train:
            for sample in subset:
                l = len(sample)
                training_samples.append(list(sample)[0:l-1])
                trainging_labels.append(sample[-1])

        for subset in test:
            l = len(subset)
            testing.append(list(subset)[0:l-1])
            testing_labels.append(subset[-1])
            
        clf = svm.SVC(decision_function_shape='ovo',kernel='sigmoid',coef0=0)
        clf.fit(training_samples, trainging_labels)
        pre = clf.predict(testing)
        mat = confusion_matrix(testing_labels,pre)
        accuracy = accuracy_score(testing_labels,pre)
        print('%dth bin is the test bin',i)
        print(mat)
        print('accuracy is: ',accuracy)
        ac += accuracy
    print("average accuracy is: ",ac/n_fold)

In [252]:
SVM(k_one_class,k_neg_class,k_zero_class,10)

%dth bin is the test bin 0
[[60  0 10]
 [ 4  0  6]
 [11  0 43]]
accuracy is:  0.7686567164179104
%dth bin is the test bin 1
[[61  1  8]
 [ 6  0  4]
 [11  0 43]]
accuracy is:  0.7761194029850746
%dth bin is the test bin 2
[[63  1  6]
 [ 4  0  6]
 [ 7  1 46]]
accuracy is:  0.8134328358208955
%dth bin is the test bin 3
[[63  0  7]
 [ 5  0  5]
 [12  0 42]]
accuracy is:  0.7835820895522388
%dth bin is the test bin 4
[[59  0 11]
 [ 4  1  5]
 [ 4  0 50]]
accuracy is:  0.8208955223880597
%dth bin is the test bin 5
[[63  0  7]
 [ 5  0  5]
 [ 6  0 48]]
accuracy is:  0.8283582089552238
%dth bin is the test bin 6
[[61  2  7]
 [ 7  0  3]
 [ 5  1 48]]
accuracy is:  0.8134328358208955
%dth bin is the test bin 7
[[62  2  6]
 [ 7  0  3]
 [ 9  0 45]]
accuracy is:  0.7985074626865671
%dth bin is the test bin 8
[[65  0  5]
 [ 6  0  4]
 [ 5  0 49]]
accuracy is:  0.8507462686567164
%dth bin is the test bin 9
[[64  2  6]
 [ 6  0  7]
 [ 8  0 54]]
accuracy is:  0.8027210884353742
average accuracy is:  0.805645